<a href="https://colab.research.google.com/github/menhguin/semantic-attention/blob/main/semantic_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install dependencies
!pip install -q transformers vllm scipy
!git clone https://github.com/menhguin/Monet.git
%cd Monet
!pip install -q .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 

In [ ]:
# Cell 2: Basic Transformers test
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "MonetLLM/monet-vd-1.4B-100BT-hf"  # Try other models if needed

# Basic generation test
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs = tokenizer("The capital of France is", return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0]))

config.json:   0%|          | 0.00/945 [00:00<?, ?B/s]

modeling_monet.py:   0%|          | 0.00/27.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/MonetLLM/monet-vd-1.4B-100BT-hf:
- modeling_monet.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/988 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


<s> The capital of France is Paris. The city is located in the north-east of France. It is the most populous


In [ ]:
# Cell 3: Expert routing inspection (corrected)
inputs = tokenizer("Artificial intelligence is", return_tensors="pt").to(model.device)
outputs = model(**inputs, output_router_probs=True)

if outputs.router_probs:
    # Get routing probabilities
    g1, g2 = outputs.router_probs[0][0], outputs.router_probs[0][1]  # shapes [batch, seq_len, heads, experts]

    # Calculate activated experts (where both g1 and g2 have >0 probability)
    activated_experts = (g1 > 0) & (g2 > 0)

    # Compute statistics
    experts_per_token = activated_experts.sum(dim=-1).float().mean()  # Average per head
    total_experts_per_token = experts_per_token * model.config.moe_heads  # Multiply by number of heads

    print(f"Average activated experts per token: {total_experts_per_token:.1f}")
else:
    print("No router probabilities found")

Average activated experts per token: 6.5


In [ ]:
# Cell 4: Expert routing inspection
# Get expert activation patterns
inputs = tokenizer("Artificial intelligence is", return_tensors="pt").to(model.device)
outputs = model(**inputs, output_router_probs=True)

# Show expert activation stats
if outputs.router_probs:
    g1, g2 = outputs.router_probs[0][0], outputs.router_probs[0][1]
    print(f"Routing probabilities shape: {g1.shape}, {g2.shape}")
    print(f"Average active experts per token: {(g1 > 0.01).float().mean().item():.2f}")
else:
    print("No router probabilities found - check model configuration")

Routing probabilities shape: torch.Size([1, 6, 8, 512]), torch.Size([1, 6, 8, 512])
Average active experts per token: 0.01


In [ ]:
# Cell 3: Replicate README example
model_name = "MonetLLM/monet-vd-1.4B-100BT-hf"  # Verify correct model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Use exact input from README
inputs = tokenizer("City and County of San Francisco", return_tensors="pt").to(model.device)
outputs = model(**inputs, output_router_probs=True)

# Expert analysis
if outputs.router_probs:
    g1, g2 = outputs.router_probs[0][0], outputs.router_probs[0][1]
    activated = (g1 > 1e-3) & (g2 > 1e-3)

    print("=== Model Config ===")
    print(f"moe_heads: {model.config.moe_heads}")
    print(f"moe_experts: {model.config.moe_experts}")
    print(f"moe_topk: {model.config.moe_topk}")

    print("\n=== Per-Token Activation ===")
    for idx, token_id in enumerate(inputs.input_ids[0]):
        token = tokenizer.decode(token_id)
        count = activated[0, idx].sum().item()
        print(f"Token {idx+1} ('{token}'): {count} experts")
else:
    print("No router probs")

=== Model Config ===
moe_heads: 8
moe_experts: 512
moe_topk: 8

=== Per-Token Activation ===
Token 1 ('<s>'): 0 experts
Token 2 ('City'): 6 experts
Token 3 ('and'): 0 experts
Token 4 ('County'): 4 experts
Token 5 ('of'): 0 experts
Token 6 ('San'): 5 experts
Token 7 ('Francisco'): 5 experts


In [ ]:
# Cell 3: Detailed expert routing analysis
if outputs.router_probs:
    g1, g2 = outputs.router_probs[0][0], outputs.router_probs[0][1]
    batch_size, seq_len, num_heads, num_experts = g1.shape

    # Calculate activated experts (where both gates have non-zero prob)
    activated = (g1 > 1e-3) & (g2 > 1e-3)

    print("\n=== Expert Activation Analysis ===")
    print(f"Model configuration:")
    print(f"- MOE heads: {model.config.moe_heads}")
    print(f"- MOE experts: {model.config.moe_experts}")
    print(f"- MOE topk: {model.config.moe_topk}")
    print(f"- MOE groups: {model.config.moe_groups}")

    print("\nPer-token statistics:")
    for i in range(seq_len):
        token = tokenizer.decode(inputs.input_ids[0][i])
        total_activated = activated[0,i].sum().item()
        print(f"Token {i+1} ('{token}'):")
        print(f"- Total activated experts: {total_activated}")
        print(f"- Experts per head: {total_activated/num_heads:.1f}")

    print("\nActivation pattern examples (first 5 experts):")
    print("Head\tExpert\tg1\t\tg2")
    for head_idx in range(2):  # Show first 2 heads
        for expert_idx in range(5):
            g1_val = g1[0,0,head_idx,expert_idx].item()
            g2_val = g2[0,0,head_idx,expert_idx].item()
            if g1_val > 0.1 or g2_val > 0.1:
                print(f"{head_idx}\t{expert_idx}\t{g1_val:.4f}\t{g2_val:.4f}")
else:
    print("No router probabilities available")


=== Expert Activation Analysis ===
Model configuration:
- MOE heads: 8
- MOE experts: 512
- MOE topk: 8
- MOE groups: 4

Per-token statistics:
Token 1 ('<s>'):
- Total activated experts: 0
- Experts per head: 0.0
Token 2 ('Art'):
- Total activated experts: 5
- Experts per head: 0.6
Token 3 ('ific'):
- Total activated experts: 9
- Experts per head: 1.1
Token 4 ('ial'):
- Total activated experts: 7
- Experts per head: 0.9
Token 5 ('intelligence'):
- Total activated experts: 10
- Experts per head: 1.2
Token 6 ('is'):
- Total activated experts: 0
- Experts per head: 0.0

Activation pattern examples (first 5 experts):
Head	Expert	g1		g2


In [ ]:
print(f"Model name: {model_name}")
print(f"moe_topk: {model.config.moe_topk} (should be 32)")
print(f"moe_experts: {model.config.moe_experts} (should be 512)")
print(f"moe_heads: {model.config.moe_heads} (should be 8)")

Model name: MonetLLM/monet-vd-1.4B-100BT-hf
moe_topk: 8 (should be 32)
moe_experts: 512 (should be 512)
moe_heads: 8 (should be 8)


In [ ]:
# Cell 3: vLLM test (corrected per README)
from vllm import LLM, SamplingParams, ModelRegistry  # Correct import per README
from Monet.modeling_monet_vllm import MonetForCausalLM

# Register custom model as shown in README
ModelRegistry.register_model("MonetForCausalLM", MonetForCausalLM)

# vLLM generation
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, max_tokens=25)
llm = LLM(
    model="MonetLLM/monet-vd-1.4B-100BT-hf",
    trust_remote_code=True,
    dtype="bfloat16",
    gpu_memory_utilization=0.85
)

outputs = llm.generate(["The meaning of life is"], sampling_params)
for output in outputs:
    print(output.outputs[0].text)

INFO 01-29 16:46:19 __init__.py:183] Automatically detected platform cuda.


ImportError: cannot import name 'get_compressed_tensors_cache_scale' from 'vllm.model_executor.layers.quantization.compressed_tensors.utils' (/usr/local/lib/python3.11/dist-packages/vllm/model_executor/layers/quantization/compressed_tensors/utils.py)